# Source localization pipeline - Stage1
- freesurfer
- .fif-File preparations
 
 


#### Author
Rudi Kreidenhuber <Rudi.Kreidenhuber@gmail.com>
#### License
BSD (3-clause)

## Subject selection

In [9]:

from src import SubjectDropDowner
import os
import glob
import mne

input_dir = "..\\..\\MEG_playground"

def get_subject_list(input_dir = input_dir) -> list:
    analist = glob.glob(os.path.join(input_dir, "*"))
    analist = [os.path.basename(f) for f in analist if os.path.isdir(f)]
    fiflist = [os.path.basename(f) for f in glob.glob(os.path.join(input_dir, "*trans_tsss.fif"))]
    fiflist = [f.split("_")[0] for f in fiflist]
    subjectlist = set(fiflist + analist)
    subjectlist = sorted([f for f in subjectlist])
    return subjectlist

subjectlist = get_subject_list()

dl = SubjectDropDowner.SubjectDropDowner(subjectlist)
drop_menu = dl.create_subject_dropdown_widget()
print(f"\n\nThe following patients/ subjects are available:")
drop_menu



The following patients/ subjects are available:


# .fif - File preprocessing
- Files are being filtered, downsampled and concatenated
- Files without head transposition and artifact correction via tsss are omitted

In [10]:
subject = dl.get_subject_dropdown().value

if not subject.startswith("sub-"):
    ject = subject
    subject = "sub-" + subject
else:
    subject = subject
    ject = subject.replace("sub-", "")

if not os.path.isdir(os.path.join(input_dir, "processed")):
    os.mkdir(os.path.join(input_dir, "processed"))

output_dir = os.path.join(input_dir, "processed")
target_file = os.path.join(output_dir, str(subject + "_prepped.fif"))

# Make sure we haven't already processed this subject
if os.path.isfile(target_file):
    print(f"\n\nThe file {target_file} already exists, aborting.\n\n")
    raise(FileExistsError)

# Configuration 
# Filter and resample
l_freq: float = 0.1         # lower pass-band edge
h_freq: float = 50.         # higher pass-band edge
fir_design: str = "firwin"  # Filter design method
s_freq: int = 300           # target sampling frequency
# ECG artifact correction
ecg_channel = "ECG003"
n_grad: int =1
n_mag: int = 1
n_eeg: int = 1


# Get all .fif files of the subject
raws = glob.glob(input_dir + "*_trans_tsss.fif")
raws = [f for f in raws if ject in f]
print(f"The following raw files were found for preprocessing:\n{raws}")
"""
prep_raws = []
for r in raws:
    raw = mne.io.read_raw(r)
    # filter
    raw.load_data()
    raw = raw.filter(l_freq, h_freq, fir_design=fir_design)
    # downsample
    if not raw.info["sfreq"] == s_freq:
        raw = raw.resample(s_freq)
    prep_raws.append(raw)
    del raw
"""
# concatenate
try:
    raw = mne.concatenate_raws(prep_raws)
    raw.save(target_file, overwrite=False)
except Exception as e:
    print(f"\n\nSomething went wrong, when trying to concatenate the raw files:\n\nError: {e}")
    




The file ..\..\MEG_playground\processed\sub-KA18051991_prepped.fif already exists, aborting.




FileExistsError: 

# Freesurfer and hippocampal segmentation

In [27]:
def get_dicom_file(subject) -> str:
    if not os.path.isdir(os.path.join(input_dir, subject)):
        print(f"No anatomical data found for {subject}, aborting\n\n")
        raise(FileNotFoundError)
    else:
        dicom_path = os.path.join(input_dir, subject, "1*", "100*", "1*", "*")
        dicom = str(glob.glob(dicom_path, recursive=True)[0])
        return dicom

def path_to_wsl(file) -> str:
    file = file.replace("\\", "/")
    return file.replace("c:", "/mnt/c")

def get_dicom_path(subject) -> str:
    dicom = get_dicom_file(subject)
    dicom = path_to_wsl(dicom)
    return dicom

def get_recon_all_command(ject) -> str:
    dicom = get_dicom_path(ject)
    subject = "sub-" + ject
    command = f"recon-all -s {subject} -i {dicom} -all && segmentHA_T1.sh {subject}"
    return command

command = get_recon_all_command(ject)

print(f"\n\nExecute the following command in the WSL shell:\n \
      (This will take hours...)\n\n{command}\n\n")



Execute the following command in the WSL shell:
       (This will take hours...)

recon-all -s sub-KA18051991 -i ../../MEG_playground/KA18051991/1000017E/1000017F/10000180/10000181 -all && segmentHA_T1.sh sub-KA18051991


